In [2]:
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Using residual connection for prediction
The idea behind this is that I will replicate ResNet to do classification. Instead of using CNN Block, I will simply use a Linear layer.

## Prepare the dataset

In [4]:
train_dataset = pd.read_csv('data/train_dataset.csv')
test_dataset = pd.read_csv('data/test_dataset.csv')

data_columns = ["V"+str(i) for i in range(1,29)]+["Amount"]
label_column ="Class"

X_train = train_dataset[data_columns]
X_test  = test_dataset[data_columns]

y_train = train_dataset[label_column]
y_test  = test_dataset[label_column]

In [6]:
x_train_tensor = torch.from_numpy(X_train.values).to(device)
y_train_tensor = torch.from_numpy(y_train.values).to(device)

x_test_tensor = torch.from_numpy(X_test.values).to(device)
y_test_tensor = torch.from_numpy(y_test.values).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

Train_dataset = DataLoader(Train_tensor, batch_size=512, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=512, shuffle=True)

## RESNET

The resnet architecture consists of multiple Residual Block, where each Residual Block is basicly the skip connection, follow the formula:

$$
    output = \mathcal{F}(input) + input
$$

### A Residual Block 

In [7]:
class Residual(nn.Module):
    def __init__(self, in_features, out_features, activation=nn.ReLU()):
        super(Residual, self).__init__()
        self.activation = activation
        self.layer1 = nn.Linear(in_features, out_features)
        self.layer2 = nn.Linear(out_features, out_features)

    def forward(self, x):
        return self.layer2(self.activation(self.layer1(x))) + x

#### Example of resnet
![Resnet](https://www.researchgate.net/publication/349717475/figure/fig4/AS:996933933993986@1614698980245/The-architecture-of-ResNet-50-model.ppm)

### Resnet 

In [8]:
class ResidualNet(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, num_residuals):
        super(ResidualNet, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(in_features, hidden_features)])
        self.layers.extend([Residual(hidden_features, hidden_features) for _ in range(num_residuals)])
        self.layers.append(nn.Linear(hidden_features, out_features))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### Parameter

In [11]:
epochs = 100
num_residual_layers = 5

net = ResidualNet(29, 1, 64, num_residual_layers).to(device)

opitmizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

### Training the model

In [12]:
for epoch in range(epochs):
    for x, y in tqdm(Train_dataset):
        opitmizer.zero_grad()
        output = net(x.float())
        loss = criterion(output, y.float().view(-1, 1))
        loss.backward()
        opitmizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in Test_dataset:
            output = net(x.float())
            predicted = (output >= 0.5).float()
            total += y.size(0)
            correct += (predicted == y.float().view(-1, 1)).sum().item()
        print(f"Epoch {epoch} Accuracy: {correct*100/total:.4f}%")

100%|██████████| 889/889 [00:12<00:00, 69.44it/s]


Epoch 0 Accuracy: 99.8268%


100%|██████████| 889/889 [00:12<00:00, 71.31it/s]


Epoch 1 Accuracy: 99.8751%


100%|██████████| 889/889 [00:13<00:00, 67.04it/s]


...

Epoch 15 Accuracy: 99.8971%


100%|██████████| 889/889 [00:13<00:00, 64.05it/s]


Epoch 16 Accuracy: 99.9402%


 54%|█████▍    | 481/889 [00:06<00:05, 71.20it/s]


KeyboardInterrupt: 

In [ ]:
# ???? 99.97% accuracy